In [1]:
%load_ext autoreload
%autoreload 2
import ML
import torch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm

In [ ]:
val_doc, test_doc = [], []
for rand in tqdm(range(10)):
    val_result, test_result, data_nums = [], [], []
    for iteration in tqdm(range(9)):
        repeat_results = []
        best = -1000
        for repeat in (range(1)):
            temp = 0
            folder = f"prepared_data/prepared_data-{rand + 1}/Scaled/Chicago_Regional/{int((iteration + 1) * 10)}"
            miss = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
            data = ML.DataLoader_Me(folder, miss[iteration], batch_size=512, device="cpu")
            train, val, test = data.train, data.val, data.test

            input_size = 3
            hidden_units = 8
            num_layer = 2
            output_size = 1
            epochs = 500
            batch_size = 32
            # learning_rate = np.random.choice(np.arange(0.0001, 0.0011, 0.0001))
            learning_rate = 0.00001

            model = ML.FlowPredict(input_size, hidden_units, output_size, lambda_value=0.01)
            model, [count_epoch, loss_values, val_loss_values] = ML.train_model(model, train,
                                                                                val, epochs, learning_rate)
            [val_rmse, val_mae, val_r2], data_val = ML.evaluate_model(model, val)
            [test_rmse, test_mae, test_r2], data_test = ML.test_model(model, test)
            if best < test_r2 or repeat == 99:
                # best = test_r2
                best_list = [[model, [count_epoch, loss_values, val_loss_values]],
                             [[val_rmse, val_mae, val_r2], data_val], [[test_rmse, test_mae, test_r2], data_test],
                             [{"input_size": [input_size], "output_size": [output_size], "hidden_units": [hidden_units],
                               "num_hidden_layer": [num_layer], "epochs": [epochs], "batch_size": [batch_size],
                               "learning_rate": [learning_rate], "optimizer": ["Adam"], "loss": ["MSE"],
                               "activation": ["ReLU"]}]]
                # break

        model, [count_epoch, loss_values, val_loss_values] = best_list[0]
        [val_rmse, val_mae, val_r2], data_val = best_list[1]
        [test_rmse, test_mae, test_r2], data_test = best_list[2]
        model_hyperparameter = pd.DataFrame(best_list[3][0])

        val_result.append([np.array(val_rmse), np.array(val_mae), val_r2])
        test_result.append([np.array(test_rmse), np.array(test_mae), test_r2])

        print(f"R2 is in{rand + 1}/10 and {iteration + 1}/9 == {test_r2}")
        ML.plot_fn(data_test, save=True, show=False, model=f"Chicago_Regional{rand}_{iteration}",
                   bins=np.arange(-0.001, 0.0011, 0.0001))

        if not os.path.exists(f"data/Result{rand + 1}/Chicago_Regional_data"):
            os.makedirs(f"data/Result{rand + 1}/Chicago_Regional_data")
        model_hyperparameter.to_csv(f"data/Result{rand + 1}/Chicago_Regional_data/HyperParameters{iteration+1}_NN_model.csv")
        data_test = np.array(data_test)
        if not os.path.exists(f"data/Result{rand + 1}/Chicago_Regional_data/data_NN_model.csv"):
            data_test = pd.DataFrame({"Predict1": data_test[0].squeeze(), "Real1": data_test[1].squeeze()})
            data_test.to_csv(f"data/Result{rand + 1}/Chicago_Regional_data/data_NN_model.csv")
        else:
            data_last = pd.read_csv(f"data/Result{rand + 1}/Chicago_Regional_data/data_NN_model.csv")
            data_last = data_last.iloc[:, 1:]
            data_last[f"Predict{iteration + 1}"] = data_test[0].squeeze()
            data_last[f"Real{iteration + 1}"] = data_test[1].squeeze()
            data_last.to_csv(f"data/Result{rand + 1}/Chicago_Regional_data/data_NN_model.csv")

        with torch.inference_mode():
            # plt.clf()
            # plt.plot(count_epoch[5:], loss_values[5:], c="b", label="Train")
            # plt.plot(count_epoch[5:], val_loss_values[5:], c="r", label="val")
            # plt.legend()
            # plt.show()
            if not os.path.exists("Models/Chicago_Regional"):
                os.makedirs("Models/Chicago_Regional")
            torch.save(model, f"Models/Chicago_Regional/Chicago_Regional{rand}_{iteration}_model_nn.pth")

    if not os.path.exists(f"data/Result{rand + 1}/Chicago_Regional_data/NN_model.csv"):
        test_result = pd.DataFrame(test_result, columns=["RMSE", "MAE", "R^2"])
        test_result.to_csv(f"data/Result{rand + 1}/Chicago_Regional_data/NN_model.csv")
    else:
        data_last = pd.read_csv(f"data/Result{rand + 1}/Chicago_Regional_data/NN_model.csv")
        data_last = data_last.iloc[:, 1:]
        test_result = pd.DataFrame(test_result, columns=["RMSE", "MAE", "R^2"])
        test_result = pd.concat([test_result, data_last], ignore_index=True)
        test_result.to_csv(f"data/Result{rand + 1}/Chicago_Regional_data/NN_model.csv")




  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

In [2]:
import LinearRegression as Ll

r2 = []
for rand in tqdm(range(10)):
    # print(f"we are going to {rand + 1} itter #########################################################################")
    all_r2 = []
    one = [0, 1, 2]
    two = [3, 4, 5]
    three = [7, 8]
    for iterr in tqdm(range(9)):
        folder = f"prepared_data/prepared_data-{rand + 1}/Scaled/Chicago_Regional/{int((iterr + 1) * 10)}"
        miss = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
        x_train, y_train, x_val, y_val, x_test, y_test = Ll.load(folder, miss[iterr])
        model = Ll.reg(x_train, y_train)
        r = Ll.testit(model, x_val, y_val)
        r2.append(r)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]